In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

pd.options.display.max_columns = 99
%matplotlib inline

loan = pd.read_csv('loans_2007.csv', nrows = 3200)
loan.info(memory_usage="deep")

#date_col = ['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3200 entries, 0 to 3199
Data columns (total 52 columns):
id                            3200 non-null int64
member_id                     3200 non-null float64
loan_amnt                     3200 non-null float64
funded_amnt                   3200 non-null float64
funded_amnt_inv               3200 non-null float64
term                          3200 non-null object
int_rate                      3200 non-null object
installment                   3200 non-null float64
grade                         3200 non-null object
sub_grade                     3200 non-null object
emp_title                     3013 non-null object
emp_length                    3110 non-null object
home_ownership                3200 non-null object
annual_inc                    3200 non-null float64
verification_status           3200 non-null object
issue_d                       3200 non-null object
loan_status                   3200 non-null object
pymnt_plan           

In [2]:
chunk_iter = pd.read_csv('loans_2007.csv', chunksize = 3000)

total_row = 0
for chunk in chunk_iter:
    total_row += len(chunk)
    print(chunk.memory_usage(deep=True).sum()/ (1024 ** 2))
    
print
print("\n"+'Total row is {}'.format(total_row))

4.614681243896484
4.6104278564453125
4.612185478210449
4.613537788391113
4.6097307205200195
4.6116132736206055
4.610204696655273
4.612573623657227
4.610699653625488
4.610750198364258
4.623462677001953
4.6223297119140625
4.62913703918457
4.8625898361206055
0.8746747970581055

Total row is 42538


3000 seems like a reasonable chunk size that will keep each below 5Mb.

# Explore columns types

In [3]:
chunk_iter = pd.read_csv('loans_2007.csv', chunksize = 3000)

for chunk in chunk_iter:
    print(chunk.dtypes.value_counts())

float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     21
int64       1
dtype: int64
float64    30
object     22
dtype: int64
float64    30
object     22
dtype: int64


# Check string columns consistency

In [4]:
chunk_iter = pd.read_csv('loans_2007.csv', chunksize = 3000)

obj_cols = []

for chunk in chunk_iter:
    chunk_obj_cols = chunk.select_dtypes(include=['object']).columns.tolist()
    if len(obj_cols) > 0:
        is_same = obj_cols == chunk_obj_cols
        if not is_same:
            print("overall obj cols:", obj_cols, "\n")
            print("chunk obj cols:", chunk_obj_cols, "\n")    
    else:
        obj_cols = chunk_obj_cols
        

overall obj cols: ['term', 'int_rate', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status', 'last_pymnt_d', 'last_credit_pull_d', 'application_type'] 

chunk obj cols: ['id', 'term', 'int_rate', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status', 'last_pymnt_d', 'last_credit_pull_d', 'application_type'] 

overall obj cols: ['term', 'int_rate', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status', 'last_pymnt_d', 'last_credit_pull_d', 'application_type'] 



# Percentage of unique values in each column

In [5]:
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000)
str_cols_vc = {}
for chunk in chunk_iter:
    str_cols = chunk.select_dtypes(include=['object'])
    for col in str_cols.columns:
        current_col_vc = str_cols[col].value_counts()
        if col in str_cols_vc:
            str_cols_vc[col].append(current_col_vc)
        else:
            str_cols_vc[col] = [current_col_vc]
            
## Combine the value counts.
combined_vcs = {}

for col in str_cols_vc:
    combined_vc = pd.concat(str_cols_vc[col])
    final_vc = combined_vc.groupby(combined_vc.index).sum()
    combined_vcs[col] = final_vc * 100 / total_row

In [6]:
# Extract useful columns for modeling
useful_obj_cols  = ['term', 'sub_grade', 'emp_title', 'home_ownership', 'verification_status', 'issue_d', 'purpose', 'earliest_cr_line', 'revol_util', 'last_pymnt_d', 'last_credit_pull_d']

for col in useful_obj_cols:
    print(col)
    print(combined_vcs[col].head(5))
    print("-----------")

term
 36 months    74.131365
 60 months    25.861583
Name: term, dtype: float64
-----------
sub_grade
A1    2.684658
A2    3.573276
A3    4.285580
A4    6.829188
A5    6.565894
Name: sub_grade, dtype: float64
-----------
emp_title
  old palm inc                       0.002351
 Brocade Communications              0.002351
 CenturyLink                         0.002351
 Department of Homeland Security     0.002351
 Down To Earth Distributors, Inc.    0.002351
Name: emp_title, dtype: float64
-----------
home_ownership
MORTGAGE    44.569561
NONE         0.018807
OTHER        0.319714
OWN          7.642578
RENT        47.442287
Name: home_ownership, dtype: float64
-----------
verification_status
Not Verified       44.097043
Source Verified    24.227749
Verified           31.668156
Name: verification_status, dtype: float64
-----------
issue_d
Apr-2008    0.608867
Apr-2009    0.782829
Apr-2010    2.143965
Apr-2011    3.674362
Aug-2007    0.173962
Name: issue_d, dtype: float64
-----------
purpo

In [7]:
# Convert columns into category type

convert_col_dtypes = {
    "sub_grade": "category", "home_ownership": "category", 
    "verification_status": "category", "purpose": "category"
}

# Process 'term' and 'revol_util' columns: remove extra characters and convert into numeric dtype

In [8]:
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000, usecols=useful_obj_cols , dtype=convert_col_dtypes, parse_dates=["issue_d", "earliest_cr_line", "last_pymnt_d", "last_credit_pull_d"])

for chunk in chunk_iter:
    chunk['term'] = chunk['term'].str.lstrip(' ').str.rstrip('months')
    chunk['term'] = pd.to_numeric(chunk['term'], downcast='integer')
    chunk['revol_util'] = chunk['revol_util'].str.lstrip(' ').str.rstrip('%')
    chunk['revol_util'] = pd.to_numeric(chunk['revol_util'], downcast='float')
    print(chunk.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 11 columns):
term                   3000 non-null int8
sub_grade              3000 non-null category
emp_title              2829 non-null object
home_ownership         3000 non-null category
verification_status    3000 non-null category
issue_d                3000 non-null datetime64[ns]
purpose                3000 non-null category
earliest_cr_line       3000 non-null datetime64[ns]
revol_util             3000 non-null float32
last_pymnt_d           2998 non-null datetime64[ns]
last_credit_pull_d     3000 non-null datetime64[ns]
dtypes: category(4), datetime64[ns](4), float32(1), int8(1), object(1)
memory usage: 339.8 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 3000 to 5999
Data columns (total 11 columns):
term                   3000 non-null int8
sub_grade              3000 non-null category
emp_title              2767 non-null object
home_ownership         3000 n

# Process Missing Values

In [9]:
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000,  dtype=convert_col_dtypes, parse_dates=["issue_d", "earliest_cr_line", "last_pymnt_d", "last_credit_pull_d"])

#usecols=useful_obj_cols ,

count_mv = {}

for chunk in chunk_iter:
    chunk['term'] = chunk['term'].str.lstrip(' ').str.rstrip('months')
    chunk['term'] = pd.to_numeric(chunk['term'], downcast='integer')
    chunk['revol_util'] = chunk['revol_util'].str.lstrip(' ').str.rstrip('%')
    chunk['revol_util'] = pd.to_numeric(chunk['revol_util'], downcast='float')
    float_col = chunk.select_dtypes(include=['float']).columns.tolist()
    for col in float_col:
        missing = len(chunk) - chunk[col].count()
        if col in count_mv:
            count_mv[col] += missing
        else:
            count_mv[col] = missing
            
count_mv

{'member_id': 3,
 'loan_amnt': 3,
 'funded_amnt': 3,
 'funded_amnt_inv': 3,
 'installment': 3,
 'annual_inc': 7,
 'dti': 3,
 'delinq_2yrs': 32,
 'inq_last_6mths': 32,
 'open_acc': 32,
 'pub_rec': 32,
 'revol_bal': 3,
 'total_acc': 32,
 'out_prncp': 3,
 'out_prncp_inv': 3,
 'total_pymnt': 3,
 'total_pymnt_inv': 3,
 'total_rec_prncp': 3,
 'total_rec_int': 3,
 'total_rec_late_fee': 3,
 'recoveries': 3,
 'collection_recovery_fee': 3,
 'last_pymnt_amnt': 3,
 'collections_12_mths_ex_med': 148,
 'policy_code': 3,
 'acc_now_delinq': 32,
 'chargeoff_within_12_mths': 148,
 'delinq_amnt': 32,
 'pub_rec_bankruptcies': 1368,
 'tax_liens': 108,
 'term': 3}

In [11]:
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000,  dtype=convert_col_dtypes, parse_dates=["issue_d", "earliest_cr_line", "last_pymnt_d", "last_credit_pull_d"])

#usecols=useful_obj_cols ,

count_mv_after = {}

for chunk in chunk_iter:
    chunk['term'] = chunk['term'].str.lstrip(' ').str.rstrip('months')
    chunk['term'] = pd.to_numeric(chunk['term'], downcast='integer')
    chunk['revol_util'] = chunk['revol_util'].str.lstrip(' ').str.rstrip('%')
    chunk['revol_util'] = pd.to_numeric(chunk['revol_util'], downcast='float')
    chunk = chunk.dropna(axis=0, how='any')
    float_col = chunk.select_dtypes(include=['float']).columns.tolist()
    for col in float_col:
        missing = len(chunk) - chunk[col].count()
        if col in count_mv_after:
            count_mv_after[col] += missing
        else:
            count_mv_after[col] = missing
            
count_mv_after

{'member_id': 0,
 'loan_amnt': 0,
 'funded_amnt': 0,
 'funded_amnt_inv': 0,
 'installment': 0,
 'annual_inc': 0,
 'dti': 0,
 'delinq_2yrs': 0,
 'inq_last_6mths': 0,
 'open_acc': 0,
 'pub_rec': 0,
 'revol_bal': 0,
 'total_acc': 0,
 'out_prncp': 0,
 'out_prncp_inv': 0,
 'total_pymnt': 0,
 'total_pymnt_inv': 0,
 'total_rec_prncp': 0,
 'total_rec_int': 0,
 'total_rec_late_fee': 0,
 'recoveries': 0,
 'collection_recovery_fee': 0,
 'last_pymnt_amnt': 0,
 'collections_12_mths_ex_med': 0,
 'policy_code': 0,
 'acc_now_delinq': 0,
 'chargeoff_within_12_mths': 0,
 'delinq_amnt': 0,
 'pub_rec_bankruptcies': 0,
 'tax_liens': 0,
 'term': 0}